In [2]:
import numpy as np
import pandas as pd
import re
import os
import itertools
import copy
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
import collections
from collections import defaultdict
import csv

In [3]:
cons = ['ɡ̠ɣ̠', 't̠ɬ̠', 'k̟x̟', 'g̟ɣ̟', 'ŋ̟.1', 'd̠ɮ̠', 'k̠x̠', 'd͡ʑ', 'lò', 'kp', 'ɡb', 'pt', 'bd', 'dʒ', 'dz', 'dɮ', 'tʃ', 'ts', 'tɬ', 'ʈʂ', 'ɖʐ', 'pf', 'bv', 'pɸ', 'bβ', 'tθ', 'dð', 'cç', 'ɟʝ', 'kx', 'ɡɣ', 'qχ', 'ɢʀ', 'ts.1', 'tɬ.1', 'dz.1', 'dɮ.1', 'œ.1', 'ʟ̠', 'ŋ̟', 'ʟ̟', 'ɣ̟', 'x̟', 'k̟', 'ɡ̟', 'ɣ̠', 'x̠', 'k̠', 'ɡ̠', 'ɰ̠', 'ɴ', 'ʀ', 'ɲ', 'ʎ', 'ŋ', 'ʟ', 'ɳ', 'ʙ', 'ɭ', 'ä', 'ɻ', 'ɽ', 'r', 'n', 'm', 'l', 'ɾ', 'ɱ', 'ʔ', 'ħ', 'ʕ', 'ʁ', 'q', 'χ', 'ɢ', 'ɕ', 'ɟ', 'ʝ', 'c', 'ç', 'tɕ', 'ɣ', 'x', 'k', 'ɡ', 'ʑ', 'ʈ', 'ɖ', 'ɬ', 'ʐ', 'ɸ', 'ʂ', 'ʒ', 'z', 'v', 't', 'ʃ', 's', 'p', 'f', 'd', 'b', 'θ', 'ɮ', 'ð', 'β', 'ɧ', 'ɰ', 'w', 'ɥ', 'j', 'ɹ', 'ʋ', 'ʍ', 'ɦ', 'h']
diacrit = ['ⁿ', 'ʲ', 'ʷ', '̯', '̥', '̬', '̹', '̜', '̩', '͡', '͜', '̊', '̃', '_', 'ʰ', '̟', '̠',  '̘', '̙', 'ʼ', 'ˤ']
vows = 'ɒaɶɑœÃɔoɤɘœəeɞøɛɵɯuʊɨʉyiʏɪ'

reg = 'ɡ̠ɣ̠|t̠ɬ̠|k̟x̟|g̟ɣ̟|ŋ̟.1|d̠ɮ̠|k̠x̠|d͡ʑ|lò|kp|ɡb|pt|bd|dʒ|dz|dɮ|tʃ|ts|tɬ|ʈʂ|ɖʐ|pf|bv|pɸ|bβ|tθ|dð|cç|ɟʝ|kx|ɡɣ|qχ|ɢʀ|ts.1|tɬ.1|dz.1|dɮ.1|œ.1|ʟ̠|ŋ̟|ʟ̟|ɣ̟|x̟|k̟|ɡ̟|ɣ̠|x̠|k̠|ɡ̠|ɰ̠|ɒ|a|ɶ|ɑ|œ|Ã|ɔ|o|ɤ|ɘ|ə|e|ɞ|ø|ɛ|ɵ|ɯ|u|ʊ|ɨ|ʉ|y|i|ʏ|ɪ|ɴ|ʀ|ɲ|ʎ|ŋ|ʟ|ɳ|ʙ|ɭ|ä|ɻ|ɽ|r|n|m|l|ɾ|ɱ|ʔ|ħ|ʕ|ʁ|q|χ|ɢ|ɕ|ɟ|ʝ|c|ç|tɕ|ɣ|x|k|ɡ|ʑ|ʈ|ɖ|ɬ|ʐ|ɸ|ʂ|ʒ|z|v|t|ʃ|s|p|f|d|b|θ|ɮ|ð|β|ɧ|ɰ|w|ɥ|j|ɹ|ʋ|ʍ|ɦ|h'


In [4]:
def truncate(n, decimals=0):
    multiplier = 10 ** decimals
    return int(n * multiplier) / multiplier

In [159]:
truncate(1, 4)

1.0

In [6]:
col_ind = ftable.index.tolist()
n_col = len(col_ind)
dist_matrix = pd.DataFrame(np.zeros((n_col, n_col)),
                               index=col_ind, columns=col_ind)
    
for i in itertools.combinations_with_replacement(ftable.index, 2):
    
    a = ftable.loc[i[0]]
    b = ftable.loc[i[1]]
    
    result = sound_dist(a, b)
    result = truncate(result, 5)
    
    dist_matrix[i[0]][i[1]] = result
    dist_matrix[i[1]][i[0]] = result

In [7]:
dist_matrix.to_csv('/Users/Stoneberry/Desktop/курсач/4/dist_matrix2.csv', sep=';')

In [5]:
non_ls_dist = pd.read_csv('/Users/Stoneberry/Desktop/курсач/4/dist_matrix2.csv', sep=';')
non_ls_dist = non_ls_dist.set_index('Unnamed: 0')
non_ls_dist.index = non_ls_dist.columns

#ftable = pd.read_csv('data.csv')
ftable = pd.read_csv('/Users/Stoneberry/Desktop/курсач/4/features2.csv', sep=';')
ftable = ftable.set_index('Unnamed: 0')

In [8]:
class Node(object):
    def __init__(self, value=''):
        self.value = value
        self.previous = None
        self.next = None
        self.dia = {}
        self.vector = value
        self.affr = False

In [32]:
class PhoneticAlgorithmIPA:
    
    def __init__(self):
 
        self.default_feature_table()

    def transcription_splitter(self, word):
    
        global cons, vows, diacrit
    
        if word == '':
            return ''

        word = word[::-1]
        #word = re.sub('˞|ˤ|ˠ|ʱ|ʴ|ʼ|ˑ|ː|:|̤|̪|̰|̺|̼|̻|̚|̟|̠|̈|̴|̽|̝|̞|̘|̙|̆|̋|́|̄|̀|̜̏|̹', '', word)
        
        word = re.sub('˞|ˠ|ʱ|ʴ|ˑ|ː|:|̤|̪|̰|̺|̼|̻|̚|̈|̴|̽|̝|̞|̆|̋|́|̄|̀|̏', '', word)
        length = len(word)
    
        head = Node()
        current = ''
        step = 0
        answer = []

        for index, letter in enumerate(word):

            if index == 0:
                current = head
        
            if letter in ("'", 'ˌ', "ˈ"):
                raise ValueError('Stress should be marked this way: V_n, where n is the number of stressed vowels.')
    
            elif letter.isdigit():
                step = [int(letter), int(letter)]
        
                if len(word)-1-index < int(letter)+1: # тут не учитываются случаи типа x͜ⁿ_1 - надо подумать над этим
                    raise ValueError('The stress is presented incorrectly')
                if word[index+1] != '_':
                    raise ValueError('The stress is presented incorrectly')
    
            elif letter in diacrit:
        
                if index == length - 1:
                    raise ValueError('{} doest not belong to any sound'.format(letter))
        
                elif letter == 'ʷ': 
                    current.dia['round'] = '+'
        
                elif letter == 'ⁿ':
                    current.dia['nasal'] = '+'
                
                elif letter == '̃':
                    current.dia['nasal'] = '+'

                elif letter == 'ʲ':
                    current.dia['back'] = '+'
                    
                elif letter == 'ʰ':
                    current.dia['aspiration'] = '+'
            
                elif letter == '̯':
                    current.dia['syllabic'] = '-'
            
                elif letter == '̩':
                    current.dia['syllabic'] = '+'    
        
                elif letter == '̬':
                    current.dia['voice'] = '+'
            
                elif letter in ('̊', '̥'):
                    current.dia['voice'] = '-'
                    
                elif letter == '̟':
                    current.dia['advanced'] = '+'
                    
                elif letter == '̠':
                    current.dia['retracted'] = '+'
                    
                elif letter == '̘':
                    current.dia['atr'] = '+'
                    
                elif letter == '̙':
                    current.dia['rtr'] = '+'
                    
                elif letter == 'ʼ':
                    current.dia['ejective'] = '+'
                    
                elif letter == 'ˤ':
                    current.dia['pharyngealized'] = '+'
                    
                #elif letter in ('̜', '̹'):
                #    current.dia['round'] = '+'
            
                elif letter in ('͡', '͜'):
                    current.affr = True
    
                elif letter == '_':
                    continue
    
            elif letter in self.feature_table.index:
        
                current.value = letter
                current.vector = letter
        
                if current.dia != {}:
                    #print(current.dia)
                    #print(self.feature_table)
                    vector = copy.copy(self.feature_table.loc[letter])
                    for value in current.dia:
                        vector[value] = current.dia[value]
                    current.vector = vector
        
                if current.affr:
                
                    if current.value not in cons or current.previous.value not in cons:
                        raise ValueError('Wrong value for an affricate')

                    current.value = [current.value, current.previous.value]
                    current.vector = [current.vector, current.previous.vector]
                    current.previous = current.previous.previous
                    answer.pop()
        
                if step != 0:
            
                    if letter not in vows:
                        raise ValueError('A non vowel element is under stress')
                
                    current.vector = copy.copy(self.feature_table.loc[letter])
                    current.vector['stress'] = '+'

                    if step[0] == 1:
                        if step[1] != 1:
                        
                            current.value = [current.value]
                            current.vector = [current.vector]
                            prev = current.previous
                            cur = current
                        
                            for i in range(int(step[1])-1):
                                current.value.append(cur.previous.value)
                                current.vector.append(cur.previous.vector)
                                prev = prev.previous
                                cur = cur.previous
                                answer.pop()
            
                            current.previous = prev
                        step = 0
    
                    else: step[0] -= 1

                if index == length - 1 and step != 0:
                    raise ValueError('The stress is presented incorrectly')
            
                answer.append(current.vector)

                if index != length - 1:
                    cur = current
                    current.next = Node()
                    current = current.next
                    current.previous = cur
    
            else: raise ValueError('Wrong value: {}'.format(letter))
    
        return answer[::-1]
    
    
    def sound_dist(self, a, b):
        
        a = pd.get_dummies(a) 
        b = pd.get_dummies(b)
    
        together = a + b
        
        similar = 0
        uncommon = 0
        common = 0

        if '+' in together and 2 in together['+'].value_counts():
            similar += together['+'].value_counts()[2]

        if '-' in together and 2 in together['-'].value_counts():
            similar += together['-'].value_counts()[2]
    
        uncommon = len(together.loc[(together['+'] == 0) & (together['-'] == 1)])
        uncommon += len(together.loc[(together['+'] == 1) & (together['-'] == 0)])
        common = similar + len(together.loc[(together['+'] == 1) & (together['-'] == 1)])
        
        #print(similar, common, uncommon)
        
        dist = 1 - (similar / (common + (uncommon * 2)))
        return dist
    
    
    def comparison(self, a, b):
    
        if type(a) == str:
        
            if type(b) == str:  # a vs a
                dist = self.distance_matrix[a][b]
            else: # a vs stress
                dist = self.sound_dist(self.feature_table.loc[a], b)
    
        if type(a) != str:
            if type(b) == str: # stress vs a
                dist = self.sound_dist(a, self.feature_table.loc[b])
            else: # stress vs stress
                dist = self.sound_dist(a, b)
        return dist
    
    
    def dift_affr(self, a, b):
        '''
        [a] vs [a, i]
        [a] vs [stress, i]
        [a] vs [stress, stress]
        
        [a, i] vs [a, i]
    
        stress vs [stress, stress]
        stress vs [stress, None]
        stress vs [None, None]
        '''
    
        res = []

        if len(a) == len(b):
            for index, item in enumerate(a):
                dist = self.comparison(item, b[index])
                res.append(dist)
            
        else:
            for item in a:
                for itm in b:
                    dist = self.comparison(item, itm)
                    res.append(dist)
        
        return min(res) + 1
    
    
    def phone_dist(self, a, b):
    
        if type(a) == str and type(b) == str:         # a vs b
            dist = self.distance_matrix[a][b]
        
        elif type(a) == str: # a 
            if type(b) is not list:                   # a vs stress
                dist = self.sound_dist(self.feature_table.loc[a], b)
            else:                                     # a vs [a, i]
                dist = self.dift_affr([a], b)
        
        elif type(b) == str: 
            if type(a) is not list:                   # stress vs a
                dist = self.sound_dist(a, self.feature_table.loc[b])
            else:
                dist = self.dift_affr(a, [b])              # [a, i] vs a
    
        elif type(a) == list:
            if type(b) == list:                       # [] vs []
                dist = self.dift_affr(a, b) 
            else:                                     # [] vs stress   
                dist = self.dift_affr(a, [b])
    
        elif type(b) == list:                         # stress vs []
            dist = self.dift_affr(b, [a])
            
        else:                                         # stress vs stress
            dist = self.sound_dist(a, b)
        
        return dist
    
    
    def lev_distance(self, a, b):
    
        # Первыми - строчки 
        # столбики - слово b
    
        dis = np.zeros((len(a) + 1, len(b) + 1))
        i = 0
        row = 0
        col = 0

        while i < dis.size:
            
            if row == 0:
                if col != 0:
                    dis[row, col] = dis[row, col-1] + 1

            elif col == 0:
                if row != 0:
                    dis[row, col] = dis[row - 1, col] + 1
            
            elif row > 1 and col > 1 and type(a[row-1]) == type(b[col-2]) and a[row-1] == b[col-2] and type(a[row-2]) == type(b[col-1]) and a[row-2] == b[col-1]:
                dis[row, col] = dis[row - 3][col - 3] + 1
        
            else:
                dis[row, col] = np.min([dis[row, col - 1] + 1,  # левый 
                                    dis[row - 1, col - 1] + self.phone_dist(a[row-1], b[col-1]), # диаг               
                                    dis[row - 1, col] + 1]) # верхний
            col += 1
            i += 1  
            if col == len(b) + 1:
                col = 0
                row += 1
        
        return dis[len(a), len(b)]

    
    def check_data(self, data):
    
        dists = []
        
        for line in data:
            
            if len(line) != 2:
                raise ValueError('There have to be 2 columns in a line. Check your delimiter.')
            
            if line[0] == line[1]:
                dist = 0
            elif line[0] == '' and line[1] == '': 
                dist = 0
            else:
                a = self.transcription_splitter(line[0])
                b = self.transcription_splitter(line[1])
            
                if a == '': 
                    dist = len(b)
                if b == '': 
                    dist = len(a)
                else: 
                    dist = self.lev_distance(a, b)

            dists.append(dist)
    
        return dists


    def dist_matrix(self, data):
        
        col_ind = data.index.tolist()
        n_col = len(col_ind)
        matrix = pd.DataFrame(np.zeros((n_col, n_col)),
                               index=col_ind, columns=col_ind)

        for i in itertools.combinations_with_replacement(data.index, 2):

            a = data.loc[i[0]]
            b = data.loc[i[1]]

            result = self.sound_dist(a, b)
            result = truncate(result, 5)
            
            matrix[i[0]][i[1]] = result
            matrix[i[1]][i[0]] = result
        
        return matrix
    
    
    def ls_dist_matrix(self, data, irrelevant_features):
        
        if irrelevant_features != []:
            
            for i in irrelevant_features:
                if i not in self.feature_table.columns:
                    raise ValueError('Incorrect irrelevant features')
            self.features = irrelevant_features
            
        else:
            index = 0
            
           # reg = '|'.join(sorted(ftable.index, key=len, reverse=True))
        
            for line in data:
                for word in line:
                    for ph in re.findall(reg, word):
                    
                        res = set(self.feature_table.loc[ph].loc[self.feature_table.loc[ph] == 'None'].keys())
                        if index == 0: self.features = res
                        else: self.features &= res
                    index += 1

        self.feature_table = self.feature_table.drop(columns=self.features)
        self.distance_matrix = self.dist_matrix(self.feature_table)

    def change_feature_table(self, d):
        
        '''
        d = {'a' : {'feature': value}}
        d = {'a': [values]}
        d = {'feature': [values]}
        
        УЧЕСТЬ ДОБАВЛЕНИЕ НОВОГО ЗВУКА В РЕГУЛЯРНЫХ ВЫРАЖЕНИЯХ 
        
        '''
        
        global ftable
    
    
        for i in d:
            
            if isinstance(d[i], list):  # d = {'feature': [values]}
                
                for value in d[i]:
                    if value not in ('+', '-', '0'):
                        raise ValueError('Incorrect feature value')
                
                if i in ftable.columns:
                    if len(d[i]) != len(ftable.index):
                        raise ValueError('All rows have to be filed')
                    self.feature_table[i] = d[i]
                    
                elif i in ftable.index:
                    if len(d[i]) != len(self.feature_table.columns):
                        raise ValueError('All columns have to be filed')
                    self.feature_table.loc[i] = d[i]
                
                else: raise ValueError('Incorrect input data')
                    
            
            elif i in ftable.index and isinstance(d[i], dict):
                
                for feature in d[i]:
                    
                    if feature not in self.feature_table.columns: raise ValueError('Incorrect feature name')
                    if d[i][feature] not in ('+', '-', '0'):
                        raise ValueError('Incorrect feature value')
                    self.feature_table.loc[i][feature] = d[i][feature]
                    
            else: raise ValueError('Incorrect input data')

        self.distance_matrix = self.dist_matrix(self.feature_table)
        
    
    def default_feature_table(self):
        self.distance_matrix = pd.read_csv('/Users/Stoneberry/Desktop/курсач/4/dist_matrix2.csv', sep=';')
        self.distance_matrix = self.distance_matrix.set_index('Unnamed: 0')
        self.distance_matrix.index = self.distance_matrix.columns
        
       #self.feature_table = pd.read_csv('data.csv')
        self.feature_table = pd.read_csv('/Users/Stoneberry/Desktop/курсач/4/features2.csv', sep=';')
        self.feature_table = self.feature_table.set_index('Unnamed: 0')
        self.feature = set()
    
    
    def phonetic_distance(self, path, delimiter=';', typ='Non_LS', irrelevant_features=[]):
        
        global ftable, non_ls_dist
        
        if not path.endswith('.csv'):
            raise ValueError('Incorrect file type. It should be csv')
            
        if not os.path.isfile(path):
            raise ValueError('Incorrect file path')
            
        if typ not in ('LS', 'Non_LS'):
            raise ValueError('Incorrect type argument')
            
        if not isinstance(irrelevant_features, list):
            raise ValueError('Wrong irrelevant_features data type!')
        
        with open(path, 'r') as csvfile:
            reader = csv.reader(csvfile, delimiter=delimiter)
            data = list(reader)

        if typ == 'LS': self.ls_dist_matrix(data, irrelevant_features)
        
        elif typ == 'Non_LS': 
            
            if irrelevant_features != []:
                raise ValueError('If you want to delete irrelevant features, use "LS" type!')
    
        dist = self.check_data(data)

        return dist 

In [50]:
ipa = PhoneticAlgorithmIPA()
ipa.phonetic_distance('/Users/Stoneberry/Desktop/курсач/4/test3.csv')

[0.6]

In [65]:
c = ['-', '-', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '+', '+', '+', '+', '+', '+', '+', '-', '-', '-']

In [66]:
d = {'sonorant': c}
ipa.change_feature_table(d)

In [69]:
ipa.phonetic_distance('/Users/Stoneberry/Desktop/курсач/4/test3.csv')

[0.09999]